In [19]:
from datasets import load_dataset, DatasetDict

# Laden des Datasets von der bereinigten CSV-Datei
dataset = load_dataset("audiofolder", data_dir="/Users/peterkompiel/python_scripts/asr4memory/processing_files/whisper-train/_output")

# Dataset in Trainings- und Testsets aufteilen
split_dataset = dataset['train'].train_test_split(test_size=0.2)  # 20% für Testdaten

# Erstellen eines DatasetDict-Objekts
dataset_dict = DatasetDict({
    'train': split_dataset['train'],
    'test': split_dataset['test']
})

# Überprüfen des ersten Datensatzes
print(dataset_dict['train'][0])
print(dataset_dict['test'][0])

{'audio': {'path': '/Users/peterkompiel/python_scripts/asr4memory/processing_files/whisper-train/_output/data/fvv0033_01_01_audio_segment_305.wav', 'array': array([0.00712585, 0.00762939, 0.00740051, ..., 0.001297  , 0.00135803,
       0.00074768]), 'sampling_rate': 16000}, 'transcription': 'No, moi synowie byli bardzo zaskoczeni.'}
{'audio': {'path': '/Users/peterkompiel/python_scripts/asr4memory/processing_files/whisper-train/_output/data/fvv0033_01_01_audio_segment_228.wav', 'array': array([ 0.01501465,  0.01365662,  0.01359558, ..., -0.00254822,
       -0.00369263,  0.01171875]), 'sampling_rate': 16000}, 'transcription': 'Mam tam siostrę, przyśle mi może jakieś pomoce, wiadomości na ten temat, a okazało się, że ona mnie tam nawet zaprosiła na dwa miesiące,'}


In [20]:
# Gesamtes DatasetDict anzeigen
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 299
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 75
    })
})


In [21]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [22]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Polish", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
input_str = dataset_dict["train"][0]["transcription"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Input:                 No, moi synowie byli bardzo zaskoczeni.
Decoded w/ special:    <|startoftranscript|><|pl|><|transcribe|><|notimestamps|>No, moi synowie byli bardzo zaskoczeni.<|endoftext|>
Decoded w/out special: No, moi synowie byli bardzo zaskoczeni.
Are equal:             True


In [24]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Polish", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
print(dataset_dict["train"][0])

{'audio': {'path': '/Users/peterkompiel/python_scripts/asr4memory/processing_files/whisper-train/_output/data/fvv0033_01_01_audio_segment_305.wav', 'array': array([0.00712585, 0.00762939, 0.00740051, ..., 0.001297  , 0.00135803,
       0.00074768]), 'sampling_rate': 16000}, 'transcription': 'No, moi synowie byli bardzo zaskoczeni.'}


In [26]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch

In [27]:
dataset_dict = dataset_dict.map(prepare_dataset, remove_columns=dataset_dict.column_names["train"], num_proc=4)

In [28]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [29]:
model.generation_config.language = "German"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None


In [30]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [31]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [32]:
import evaluate

metric = evaluate.load("wer")

In [33]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [34]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-pl",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=False,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [35]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

max_steps is given, it will override any value given in num_train_epochs


In [36]:
trainer.train()

RuntimeError: Mismatched Tensor types in NNPack convolutionOutput